1. Install dependencies

In [103]:
!pip install torch torchvision transformers==4.36.0 datasets==3.6.0 evaluate accelerate timm

2. imports

In [104]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from datasets import load_dataset
from transformers import ViTForImageClassification, ViTImageProcessor
from tqdm import tqdm

3. Load the dataset

In [105]:
dataset = load_dataset("fcakyon/pokemon-classification", name="full", trust_remote_code=True)

4. Setup processor (preprocessing images)

In [106]:
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

5. Define transform

In [107]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std),
])

6. Dataset wrapper

In [108]:
class PokemonDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform, processor):
        self.dataset = dataset
        self.transform = transform
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        print(f"Accessing index: {idx}")
        print(f"Dataset type: {type(self.dataset)}")
        print(f"Dataset length: {len(self.dataset)}")
        item = self.dataset[idx]
        image = item["image"]   # raw PIL Image
        label = item["labels"]  # use actual label from dataset
        image = self.transform(image)
        return {"pixel_values": image, "labels": label}

7. Split into train/val

In [109]:
train_dataset = PokemonDataset(dataset["train"].select(range(1000)), transform, processor)
val_dataset   = PokemonDataset(dataset["train"].select(range(1000, 1200)), transform, processor)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=8)

8. Load pretrained ViT model

In [110]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=150,   # change according to your dataset
    ignore_mismatched_sizes=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([150]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([150, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

9. Training setup

In [111]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

10. Training loop

In [112]:
epochs = 10

for epoch in range(epochs):
    print(f"Epoch {epoch+1}")
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Training loss: {avg_loss:.4f}")

Epoch 1


  0%|          | 0/125 [00:00<?, ?it/s]

Accessing index: 833
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 430
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 532
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 999
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 455
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 488
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 2
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 620
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000


  1%|          | 1/125 [00:37<1:18:05, 37.79s/it]

Accessing index: 802
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 186
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 132
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 949
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 509
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 359
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 697
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 271
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000


...

Accessing index: 681
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 351
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 978
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 313
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 852
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 788
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 246
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 591
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000


 99%|█████████▉| 124/125 [34:34<00:16, 16.88s/it]

Accessing index: 864
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 185
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 747
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 988
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 384
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 247
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 387
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 576
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000


100%|██████████| 125/125 [34:51<00:00, 16.73s/it]

Training loss: nan


11. Save model

In [113]:
model.save_pretrained("./pokemon-vit")
processor.save_pretrained("./pokemon-vit")

['./pokemon-vit/preprocessor_config.json']

12. Load saved model and predict

In [114]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for batch in val_loader:
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values)
        preds = outputs.logits.argmax(-1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {correct/total:.2%}")

Accessing index: 0
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 1
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 2
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 3
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 4
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 5
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 6
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 7
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 8
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 9
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 10
Dataset type: <class 'datasets.arrow_dat